## FDMS TME3  

Kaggle [How Much Did It Rain? II](https://www.kaggle.com/c/how-much-did-it-rain-ii)

Florian Toque & Paul Willot  

In [1]:
%matplotlib inline
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import random
import pandas as pd

In [2]:
filename = "data/train.csv"
train = pd.read_csv(filename)


In [40]:
mini_train = pd.read_csv(filename,nrows=100)
mini_train[mini_train["Ref_5x5_10th"]]



KeyError: '[  nan   nan   nan   nan   nan   nan   5.   22.5  15.5  14.   16.5  12.\n  19.   14.   11.    3.5  14.5  16.5   nan   nan   nan   nan   nan   nan\n   nan   nan   nan   nan   nan   nan  16.   14.   24.5   nan   nan   nan\n  10.5  33.5  33.5   nan   nan   nan   8.5   nan   nan   nan   nan   nan\n   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan\n   nan   nan   nan   nan  10.5   nan   nan   nan   nan   nan   nan   nan\n   nan   nan   nan   nan   nan   nan   nan   nan  17.5   nan   nan  19.\n  22.5  22.   13.5  14.    nan   nan   nan   nan   nan   nan   nan   nan\n   nan   nan   nan   nan] not in index'

In [55]:
mini_train_clean = mini_train[[not i for i in np.isnan(mini_train["Ref_5x5_10th"])]]

In [82]:
mini_train_clean.ix(1)


In [6]:
for i in np.arange(1,10):
    print(np.random.choice(("kevin","pas kevin")))

kevin
pas kevin
kevin
kevin
kevin
pas kevin
kevin
pas kevin
kevin
